In [ ]:
# !pip install xgboost==1.4.2
# !pip install scikit-optimize

     |████████████████████████████████| 100 kB 4.3 MB/s 


In [3]:
import sys
import os
    
import numpy as np
import pandas as pd
from sklearn.model_selection import cross_val_score 
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import GridSearchCV
import joblib
from sklearn import model_selection
from sklearn import metrics
from sklearn.linear_model import LinearRegression
from sklearn import metrics
import matplotlib.pyplot as plt

from google.colab import drive
drive.mount('/content/gdrive/')

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [9]:
var = 'phosphorus'

indata = pd.read_excel(('/content/gdrive/MyDrive/SBP_vars/data_'+ var + '.xlsx'),
                       sheet_name="Sheet4",
                       thousands=(","), header=0)

labels = np.array(indata[var])

features = indata[["B02_mean", "B03_mean", "B04_mean", "B05_mean", "B06_mean",
                   "B07_mean", "B08_mean", "B09_mean", "B11_mean", "B12_mean", "B8A_mean", "NDVI",
                   "GNDVI", "SR", "NDRE", "NDWI", "PREC", "TEMP", "RAD", "Elevation", "slope", 
                   "Morfologia", "AUX", "Lat", "Lon"]]
 

features = np.array(features)


cv_m = joblib.load('/content/gdrive/MyDrive/SBP_vars/cross_validation_folds/cv_farm_' + 
                   var +'.joblib')


tm_r2_train = np.zeros((len(cv_m),1))
tm_train_rmse =np.zeros((len(cv_m),1))
tm_rrmse_train  =np.zeros((len(cv_m),1))
tm_r2_test = np.zeros((len(cv_m),1))
tm_rmse_test = np.zeros((len(cv_m),1))
tm_rrmse_test = np.zeros((len(cv_m),1))

tm_test_ypred = np.zeros((len(labels)))

for i in range(len(cv_m)):
    reg = LinearRegression()
    reg.fit(features[cv_m[i][0]], labels[cv_m[i][0]])
    tm_r2_train[i] = reg.score(features[cv_m[i][0]], labels[cv_m[i][0]])
    
    ob = labels[cv_m[i][0]]
    tm_tr_ypred = reg.predict(features[cv_m[i][0]])
    tm_train_rmse[i] = metrics.mean_squared_error(labels[cv_m[i][0]], tm_tr_ypred, squared=False)
    tm_rrmse_train[i] = tm_train_rmse[i] / np.mean(ob)
    
    tm_ypred = reg.predict(features[cv_m[i][1]])
    tm_test_ypred[cv_m[i][1]] = tm_ypred
    
    ob = labels[cv_m[i][1]]
    tm_r2_test[i] =  metrics.r2_score(ob, tm_ypred)
    tm_rmse_test[i] = metrics.mean_squared_error(ob, tm_ypred, squared=False)
    tm_rrmse_test[i] = tm_rmse_test[i] / np.mean(ob)


rr_train = np.concatenate([tm_train_rmse, tm_rrmse_test, tm_r2_train], axis=1)

rr_test = np.concatenate([tm_rmse_test, tm_rrmse_test, tm_r2_test], axis=1)

rr_train_overall = np.zeros((1,3))

rr_train_overall[0,0] = np.mean(tm_train_rmse)
rr_train_overall[0,1] = np.mean(tm_rrmse_test)
rr_train_overall[0,2] = np.mean(tm_r2_train)


rr_test_overall = np.zeros((1,3))

rr_test_overall[0,0] = metrics.mean_squared_error(labels, tm_test_ypred, squared=False)
rr_test_overall[0,1] = rr_test_overall[0,0] / np.mean(labels)
rr_test_overall[0,2] = metrics.r2_score(labels, tm_test_ypred)

results = {"results_train": rr_train, 
           "results_test": rr_test, 
           "results_train_oveall": rr_train_overall,
           "results_test_oveall": rr_test_overall
           }

a= os.path.basename(os.path.realpath(sys.argv[0]))

joblib.dump(results, '/content/gdrive/MyDrive/SBP_vars/results_farm/results_'+ 'mlr' + '_' + var + '.joblib')
print(rr_train_overall)
print(rr_test_overall)

[[0.67287957 0.31898096 0.53742892]]
[[0.86337098 0.33468161 0.24661814]]
